# Variogram Point Cloud - tutorial

## Table of Contents:

1. Read point data,
2. Create variogram point cloud,
3. Detect and remove outliers.
4. Calculate experimental semivariance from point cloud.

## Level: Basic

## Changelog

| Date       | Change description                                                                                                                                                     | Author                     |
|------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------|
| 2022-08-17 | Updated to the version 0.3.0                                                                                                                                           | @SimonMolinsky             |
| 2021-12-13 | Changed behavior of `select_values_in_range()` function                                                                                                                | @SimonMolinsky             |
| 2021-10-13 | Refactored TheoreticalSemivariogram (name change of class attribute) and refactored `calc_semivariance_from_pt_cloud()` function to protect calculations from `NaN's`. | @ethmtrgt & @SimonMolinsky |
| 2021-08-22 | Refactored the Outlier Removal algorithm - quantile based algorithm                                                                                                    | @SimonMolinsky             |
| 2021-08-10 | Refactored the Outlier Removal algorithm                                                                                                                               | @SimonMolinsky             |
| 2021-05-11 | Refactored TheoreticalSemivariogram class                                                                                                                              | @SimonMolinsky             |
| 2021-03-31 | Update related to the change of semivariogram weighting. Updated cancer rates data.                                                                                    | @SimonMolinsky             |

## Introduction

In this tutorial we will learn how to read and prepare data for semivariogram modeling, how to manually find the best step size between lags and how to detect outliers in our data which can affect final semivariogram.

Variogram Point Cloud analysis is an additional, basic data preparation step which may save you a lot of headache with more sophisticated analysis. You should learn about Variogram Point Cloud analysis before you move on to the semivariogram estimation and semivariogram fitting operations.

We use: 

- for point 1 and 2: DEM data which is stored in a file `samples/point_data/txt/pl_dem_epsg2180.txt`,
- for point 3 and 4: Breast cancer rates data is stored in the shapefile in folder `samples/regularization/cancer_data.gpkg`.

## Import packages

In [ ]:
import numpy as np

from pyinterpolate import Blocks
from pyinterpolate.io import read_txt
from pyinterpolate.distance.distance import calc_point_to_point_distance
from pyinterpolate.variogram.empirical.variogram_cloud import VariogramCloud
from pyinterpolate.variogram.empirical.experimental_variogram import calculate_semivariance

## 1) Read point data

In [ ]:
dem = read_txt('samples/point_data/txt/pl_dem_epsg2180.txt')

In [ ]:
# Look into a first few lines of data

dem[:10, :]

## 2) Set proper lag size with variogram cloud histogram

In this step we generate Variogram Point Cloud. We calculate it for 16 lags and test different cloud variogram visualization methods.

In [ ]:
# Create analysis parameters

# Check max distance between points
distances = calc_point_to_point_distance(dem[:, :-1])
maximum_range = np.max(distances) / 2

number_of_lags = 16
step_size = maximum_range / number_of_lags

vc = VariogramCloud(input_array=dem, step_size=step_size, max_range=maximum_range)

Now we check how many points are grouped for each lag:

In [ ]:
for idx, _lag in enumerate(vc.lags):
    print(f'Lag {_lag} has {vc.points_per_lag[idx]} point pairs.')

As we see, there is a lot of points per lag! Data is probably messy, but to be sure, we must check its distribution. What are those **points per lag**? It is a number set of semivariances between all point pairs within a specific lag. This information may be useful to know if there are any undersampled ranges, and we can change `step_size` accordingly (to avoid situation, where a lag has the order of magnitude less point pairs than other lags).

With the information about data density, we can take a closer look into data distribution. The `VariogramCloud` class has three types of plots that can be used for analysis:

1. Scatter plot (per lag). It shows general dispersion of semivariances, but it is hard to read more from this type of plot.
2. Box plot. It shows dispersion and quartiles of semivariances per lag. It is a great tool to detect outliers. We can check distribution differences per lag, and their deviation from normality or skewness (slightly harder task).
3. Violin plot. It is a box plot on steroids. We can read all information as from box plot plus we see a distribution of semivariances per lag.

Let's plot them all. First, is scatter plot. It can take a while (we have thousands of point pairs), so now it is a good time to prepare a beverage.

In [ ]:
vc.plot('scatter')

The output per lag is dense and we cannot distinguish a single semivariance values from a scatter plot. But, we can follow a general trend and see maxima on the plot. We can make it better, if we use a box plot instead:

In [ ]:
vc.plot('box')

The orange line in a middle of each box represents median (50%) value per lag. The trend is more visible if we look into it. What else can be seen here?

1. Dispersion of values is greater with a distance, but at the same time median and the **3rd quartile** of semivariance values is rising (**3rd quartile** is plotted as a horizontal line on a top of a box.
2. The maximum value rises with a distance (it is a horizontal line on a top of a whisker).
3. Data is skewed towards lower values of semivariance. Why? Because median is closer to the bottom of a box than to the center or the top.
4. The **1st quartile** (25% of the lowest values) rises with a lags.

We could stop our analysis here, but there is a better option to analyze data distribution than a box plot. We can use a violin plot:

In [ ]:
vc.plot('violin')

The violin plot supports our earlier ideas but it shows additional pattern that we should be aware of. For a distant lags, a distribution starts to be multimodal. We see one mode around a very low values of semivariance and another mode close to the 1st quartile. Multimodality may affect our final outcomes, and maybe it tells us that there are more than 1 levels of spatial dependency?

There is elephant in the room that we didn't cover yet. Our data is literally pulled up with outliers. Do you see long whiskers on a top of every distribution? We can let it be, but maybe it is a better idea to slightly clean those results before we start Kriging? In the next paragraph, we will detect and remove outliers from a block dataset.

## 3) Detect and remove outliers

With idea how Variogram Point Cloud works we can detect and "remove" outliers from our dataset. We use for it other dataset which represents the breast cancer rates in counties of Northeastern part of the U.S. Each county will be transformed into its centroid. Those centroids are not evenly spaced and we can expect that for some number of steps dataset may be modeled incorrectly.

In [ ]:
# Read and prepare data

DATASET = 'samples/regularization/cancer_data.gpkg'
POLYGON_LAYER = 'areas'
GEOMETRY_COL = 'geometry'
POLYGON_ID = 'FIPS'
POLYGON_VALUE = 'rate'
MAX_RANGE = 400000
STEP_SIZE = 40000

AREAL_INPUT = Blocks()
AREAL_INPUT.from_file(DATASET, value_col=POLYGON_VALUE, index_col=POLYGON_ID, layer_name=POLYGON_LAYER)

AREAL_INPUT.data.head()

In [ ]:
areal_centroids = AREAL_INPUT.data[['centroid.x', 'centroid.y', 'rate']].values

In [ ]:
# Create analysis parameters

# Check max distance between points

distances = calc_point_to_point_distance(areal_centroids[:, :-1])
maximum_range = np.max(distances) / 2

number_of_lags = 8
step_size = maximum_range / number_of_lags

vc = VariogramCloud(input_array=areal_centroids, step_size=step_size, max_range=maximum_range)

In [ ]:
vc.plot('violin')

In [ ]:
for idx, _lag in enumerate(vc.lags):
    print(f'Lag {_lag} has {vc.points_per_lag[idx]} point pairs.')

Our data is skewed towards large values, thus we will remove outliers from the upper part of dataset. We will use inter-quartile range algorithm. It detects outliers as all points below the first quartile of a data plus `m` (positive or zero `float`) standard deviations, and all points above 3rd quartile plus `n` (positive or zero `float`) standard deviations. The important thing is, that we can set different `m` and `n`, we should fit those values to a distribution and it's skewness.

The class `VariogramCloud` has internal method `.remove_outliers()`. With parameter `inplace` set to `True` we can overwrite the variogram point cloud, but we set it to `False` and it will return a new `VariogramCloud` object with a cleaned variogram point cloud. Other parameters:

* `method`: is a string with a method of removing outliers. We have two methods, one based on **z-scores** and **inter-quartile range** method. Both have upper and lower bounds. Z-score method is invoked with `method='zscore'`, and arguments for this function are `z_lower_limit` and `z_upper_limit` (number of standard deviations from the mean down and up, or negative and positive).
* `iqr_lower_limit`: how many standard deviations should be subtracted from the first quartile to set a limit of valid values,
* `iqr_upper_limit`: how many standard deviations should be added to the 3rd quartile to set a limit of valid values.

The values we will get will be within a limits:

$$q1 - (m * std) < V < q3 + (n * std),$$

where:
* $q1$: 1st quartile,
* $q3$: 3rd quartile,
* $std$: standard deviation,
* $m$: real value greater than 0,
* $n$: real value greater than 0,
* $V$: the cleaned array.

In [ ]:
cvc = vc.remove_outliers(method='iqr', iqr_lower_limit=3, iqr_upper_limit=2, inplace=False)

In [ ]:
cvc.plot('violin')

When we compare both figures - before and after data cleaning - we see that the y-axis of the second plot is 6-7 times smaller than the y-axis of the first plot! We've cleaned our data from the extreme values.

## 4) Calculate experimental semivariogram from point cloud

The last but not least property of the experimental variogram point cloud is that we may calculate semivariogram directly from it. The method for it is `.calculate_experimental_variogram()`. Let's compare outputs from two algorithms.

In [ ]:
exp_variogram_from_points = calculate_semivariance(areal_centroids, step_size=step_size, max_range=maximum_range)
exp_variogram_from_p_cloud = vc.calculate_experimental_variogram()

In [ ]:
np.array_equal(exp_variogram_from_points, exp_variogram_from_p_cloud)

Both semivariograms are the same, but methods to obtain those were different.

---

## Where to go?

### Next steps:

* [Outliers and Their Influence on the Model]()
* [Ordinary and Simple Kriging]()

### More about variogram modeling:

* [Semivariogram Estimation]()
* [Theoretical Models]()